 # Using met.no forecasts with weather-tools



 This notebook-style script shows how to fetch met.no forecasts and combine them

 with SILO observations using the new `weather_tools` APIs. Open it directly in

 JupyterLab or VS Code with Jupytext to run the cells interactively.

 ## Prerequisites

 - Network access to `https://api.met.no` and a valid contact e-mail for the User-Agent header.

 - Set `SILO_API_KEY` (your registered SILO e-mail) or pass `api_key=` directly to `SiloAPI`.

 - Install `weather-tools` in your environment (either via `pip install weather-tools` or the local checkout).

 - Optional: enable logging (e.g. `logging.basicConfig(level=logging.INFO)`) to inspect requests.

In [1]:
import pandas as pd

from weather_tools.merge_weather_data import get_merge_summary, merge_historical_and_forecast
from weather_tools.metno_api import MetNoAPI
from weather_tools.metno_models import MetNoFormat, MetNoQuery
from weather_tools.silo_api import SiloAPI
from weather_tools.silo_models import AustralianCoordinates


In [12]:
# silo_api = SiloAPI(log_level="INFO")
# station_meta = silo_api.search_stations("Northam").iloc[0]

station_meta = {'station_code': 10111,
 'name': 'NORTHAM',
 'latitude': -31.651,
 'longitude': 116.659,
 'state': 'WA',
 'elevation': 170.0}

## 1. Configure the met.no API client

 met.no requires a descriptive User-Agent string that includes contact details.

 Replace the placeholder below with your own application name and e-mail.

In [2]:
api = MetNoAPI(
    # user_agent="weather-tools-example/0.1 (contact: you@example.com)",
    enable_cache=True,
    log_level="DEBUG",
)


## 2. Build a forecast query for your location

 The `AustralianCoordinates` model validates that the latitude and longitude sit

 within the range supported by SILO (GDA94 datum). Adjust the coordinates to

 target your site of interest.

In [13]:
coordinates = AustralianCoordinates(latitude=station_meta['latitude'], longitude=station_meta['longitude'])  # Northam, WA
query = MetNoQuery(coordinates=coordinates, format=MetNoFormat.COMPACT)

response = api.query_forecast(query)
metadata = response.get_meta()

metadata


[10/24/25 11:13:31] INFO     Request successful on attempt 1

{'updated_at': '2025-10-24T01:17:07Z',
 'units': {'air_pressure_at_sea_level': 'hPa',
  'air_temperature': 'celsius',
  'cloud_area_fraction': '%',
  'precipitation_amount': 'mm',
  'relative_humidity': '%',
  'wind_from_direction': 'degrees',
  'wind_speed': 'm/s'}}

 ## 3. Convert forecasts to daily and hourly tables

 `MetNoAPI.to_dataframe` can aggregate the hourly GeoJSON payload to daily summaries

 or return hourly data. Use the `frequency` parameter to control aggregation:

 - `frequency='daily'` (default) - Daily aggregates

 - `frequency='hourly'` - Raw hourly data

 - `frequency='weekly'` or `frequency='monthly'` - Other time periods

In [14]:
daily_forecasts = api.to_dataframe(response, frequency='daily')
hourly_forecasts = api.to_dataframe(response, frequency='hourly')

daily_forecasts.head()

,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-24,7.7,20.5,0.4,5.714286,10.1,66.847619,1009.771429,27.828571,lightrain
1,2025-10-25,9.3,23.0,0.0,4.237500,5.8,64.400000,1014.687500,40.420833,partlycloudy_day
2,2025-10-26,12.6,23.9,0.0,3.042105,5.1,49.726316,1017.521053,39.263158,partlycloudy_day
3,2025-10-27,15.8,26.5,0.0,2.875000,3.5,35.650000,1016.875000,0.000000,clearsky_day
4,2025-10-28,11.9,34.5,0.0,4.325000,7.1,47.750000,1008.800000,33.000000,partlycloudy_day


In [15]:
hourly_forecasts.head()


,time,air_pressure_at_sea_level,air_temperature,cloud_area_fraction,relative_humidity,wind_from_direction,wind_speed,precipitation_amount,symbol_code
0,2025-10-24 03:00:00,1009.1,19.5,25.0,43.4,285.9,7.3,0.0,fair_day
1,2025-10-24 04:00:00,1008.6,20.2,12.5,38.1,288.6,7.8,0.0,clearsky_day
2,2025-10-24 05:00:00,1008.1,20.5,37.5,33.2,292.6,8.7,0.0,fair_day
3,2025-10-24 06:00:00,1008.0,19.5,93.7,37.0,282.7,10.1,0.1,lightrain
4,2025-10-24 07:00:00,1008.2,16.7,55.5,55.6,286.3,8.9,0.3,rainshowers_day


 ### Quick helper: `get_daily_forecast`

 `get_daily_forecast` returns a pandas DataFrame directly with daily summaries,


In [16]:
daily_df = api.get_daily_forecast(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    days=5,
)

daily_df  # Already a DataFrame!

,date,min_temperature,max_temperature,total_precipitation,avg_wind_speed,max_wind_speed,avg_relative_humidity,avg_pressure,avg_cloud_fraction,dominant_weather_symbol
0,2025-10-24,7.7,20.5,0.4,5.714286,10.1,66.847619,1009.771429,27.828571,lightrain
1,2025-10-25,9.3,23.0,0.0,4.237500,5.8,64.400000,1014.687500,40.420833,partlycloudy_day
2,2025-10-26,12.6,23.9,0.0,3.042105,5.1,49.726316,1017.521053,39.263158,partlycloudy_day
3,2025-10-27,15.8,26.5,0.0,2.875000,3.5,35.650000,1016.875000,0.000000,clearsky_day
4,2025-10-28,11.9,34.5,0.0,4.325000,7.1,47.750000,1008.800000,33.000000,partlycloudy_day


## 4. Merge met.no forecasts with SILO history

 Pull the last five days from the SILO DataDrill API for the same coordinates,

 then merge that history with the met.no forecast. The helper automatically

 converts column names, fills optional variables (when enabled), and annotates

 the data source for each record.

In [17]:
first_forecast_date = pd.to_datetime(daily_df["date"]).min()
# history_end = first_forecast_date - pd.Timedelta(days=1)
history_start = first_forecast_date - pd.Timedelta(days=4)
silo_api = SiloAPI(log_level="INFO")
silo_history = silo_api.get_gridded_data(
    latitude=coordinates.latitude,
    longitude=coordinates.longitude,
    start_date=history_start.strftime("%Y%m%d"),
    end_date=first_forecast_date.strftime("%Y%m%d"),
    variables=["rainfall", "max_temp", "min_temp", "evaporation"],
)

# silo_history = raw_silo.rename(columns={"YYYY-MM-DD": "date"}).copy()

# silo_history["date"] = pd.to_datetime(silo_history["date"])

# required_cols = ["date", "min_temp", "max_temp", "daily_rain"]
# missing = [col for col in required_cols if col not in silo_history.columns]
# if missing:
#     raise ValueError(f"Missing required columns from SILO response: {missing}")

# silo_data = silo_history[required_cols].sort_values("date").reset_index(drop=True)

merged = merge_historical_and_forecast(
    silo_data=silo_history,
    metno_data=daily_df,
    overlap_strategy="prefer_silo",
    return_cols="silo_only",
)

merged


[10/24/25 11:14:05] INFO     Request successful on attempt 1

                    INFO     Auto-detected transition date: 2025-10-24 00:00:00

                    INFO     Merged 4 SILO records with 5 met.no records. Total: 9 records

,latitude,longitude,daily_rain,max_temp,min_temp,evap_pan,date,data_source,is_forecast
0,-31.651,116.659,0.1,21.2,5.9,3.2,2025-10-20,silo,False
1,-31.651,116.659,19.6,22.4,8.6,4.5,2025-10-21,silo,False
2,-31.651,116.659,0.0,25.1,8.3,4.7,2025-10-22,silo,False
3,-31.651,116.659,0.0,25.5,10.0,5.8,2025-10-23,silo,False
4,NaN,NaN,0.4,20.5,7.7,NaN,2025-10-24,metno,True
5,NaN,NaN,0.0,23.0,9.3,NaN,2025-10-25,metno,True
6,NaN,NaN,0.0,23.9,12.6,NaN,2025-10-26,metno,True
7,NaN,NaN,0.0,26.5,15.8,NaN,2025-10-27,metno,True
8,NaN,NaN,0.0,34.5,11.9,NaN,2025-10-28,metno,True


In [19]:
merge_summary = get_merge_summary(merged)
merge_summary


{'total_records': 9,
 'silo_records': np.int64(4),
 'metno_records': np.int64(5),
 'date_range': {'start': Timestamp('2025-10-20 00:00:00'),
  'end': Timestamp('2025-10-28 00:00:00'),
  'days': 9},
 'silo_period': {'start': Timestamp('2025-10-20 00:00:00'),
  'end': Timestamp('2025-10-23 00:00:00')},
 'metno_period': {'start': Timestamp('2025-10-24 00:00:00'),
  'end': Timestamp('2025-10-28 00:00:00')},
 'transition_date': Timestamp('2025-10-24 00:00:00')}

## 4b. Merge Forecast with SILO PatchedPoint 

 Pull the last four days from the SILO Patch Point API for the same coordinates, using the station code. 


In [21]:

first_forecast_date = pd.to_datetime(daily_forecasts["date"]).min()
history_end = first_forecast_date #- pd.Timedelta(days=1) # Include day before forecast to check overlap handling
history_start = history_end - pd.Timedelta(days=4)


silo_station_data = silo_api.get_station_data(station_code=str(station_meta["station_code"]),
						  start_date=history_start.strftime("%Y%m%d"),
						end_date=history_end.strftime("%Y%m%d"),
						variables=["rainfall", "max_temp", "min_temp", "evaporation"],
		)

merge_historical_and_forecast(
    silo_data=silo_station_data,
    metno_data=daily_df,
	return_cols="silo_only",
)

[10/24/25 11:17:12] INFO     Request successful on attempt 1

                    INFO     Auto-detected transition date: 2025-10-24 00:00:00

                    INFO     Merged 4 SILO records with 5 met.no records. Total: 9 records

,station,daily_rain,max_temp,min_temp,evap_pan,date,data_source,is_forecast
0,10111.0,0.0,21.4,6.0,3.2,2025-10-20,silo,False
1,10111.0,19.8,22.5,9.0,4.5,2025-10-21,silo,False
2,10111.0,0.0,25.1,8.7,4.7,2025-10-22,silo,False
3,10111.0,0.0,25.6,10.0,0.0,2025-10-23,silo,False
4,NaN,0.4,20.5,7.7,NaN,2025-10-24,metno,True
5,NaN,0.0,23.0,9.3,NaN,2025-10-25,metno,True
6,NaN,0.0,23.9,12.6,NaN,2025-10-26,metno,True
7,NaN,0.0,26.5,15.8,NaN,2025-10-27,metno,True
8,NaN,0.0,34.5,11.9,NaN,2025-10-28,metno,True
